# Image recognition with MNIST Dataset

MNIST dataset is one of the most well studied datasets for image recognition.

**Objective**
Experiment with various deep learning techniques to see it's effects and understand what they mean.

I will be iteratively trying something and see it's effects on train accuracy and validation(test) accuracy. This is just an exercise to see how tinkering with each cog would work, and this approach is advisable only for learning purpose. 

TL;DR version

**Changes made compared to base model**

1. Increased epoch number from 10 to 20
2. Added transformation for random rotation

**Conclusion**
1. Noticed that training accuracy dropped in the beginning, but rose to base model's ccuracy at 10 epochs, after 20 epochs

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms

dtype = torch.float
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load in data

Let's load in the data. Thankfully torchvision library has inbuilt functions to download the data and load it in.
We will be doing only minimal transformations.

In [2]:
data_transforms = {
    'train' : transforms.Compose(
    [transforms.RandomRotation(20),
     transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))
    ]),
    'val' : transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))
    ])
}

img_datasets = {'train': datasets.MNIST(root='./data', train=True, download=True, transform=data_transforms['train']),
                'val' : datasets.MNIST(root='./data', train=False, download=True, transform=data_transforms['val'])
                }
    
dataloaders_dict = {'train': torch.utils.data.DataLoader(img_datasets['train'], batch_size=64, 
                                                       shuffle=True, num_workers=4),
                    'val': torch.utils.data.DataLoader(img_datasets['val'], batch_size=64, 
                                                       shuffle=False, num_workers=4)
                    }

## Network definition

Here we will be defining our network.

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        # an affine operation: y = Wx + b
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


# Training & Testing

Next we will create functions that run training and testing cycles

In [4]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_corrects = 0
    running_loss = 0.0
    #optimizer = sgdr(optimizer, epoch+1)
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        _, preds = torch.max(output, 1)
        loss.backward()
        optimizer.step()
        
        running_corrects += torch.sum(preds == target.data)
        running_loss += loss.item() * target.size(0)
    print('Train Epoch: {} Acc: {} Loss: {:.6f}'.format(epoch, running_corrects, running_loss))
            
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))

In [5]:
def run_model():
    num_epochs = 20
    learning_rate = 3e-4
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=7)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train(net, device, dataloaders_dict['train'], optimizer, criterion, epoch)
        test(net, device, dataloaders_dict['train'], criterion)
        scheduler.step()

Now let's run the model and see, what we get ...

In [6]:
run_model()

Train Epoch: 0 Acc: 42486 Loss: 54334.084827
Test set: Average loss: 0.0039, Accuracy: 55797/60000 (93%)
Train Epoch: 1 Acc: 52208 Loss: 26023.474498
Test set: Average loss: 0.0026, Accuracy: 56989/60000 (95%)
Train Epoch: 2 Acc: 53896 Loss: 20713.435229
Test set: Average loss: 0.0021, Accuracy: 57468/60000 (96%)
Train Epoch: 3 Acc: 54582 Loss: 18394.412598
Test set: Average loss: 0.0019, Accuracy: 57858/60000 (96%)
Train Epoch: 4 Acc: 55122 Loss: 16393.656138
Test set: Average loss: 0.0018, Accuracy: 57872/60000 (96%)
Train Epoch: 5 Acc: 55311 Loss: 16084.132507
Test set: Average loss: 0.0017, Accuracy: 57985/60000 (97%)
Train Epoch: 6 Acc: 55571 Loss: 15434.033919
Test set: Average loss: 0.0016, Accuracy: 58076/60000 (97%)
Train Epoch: 7 Acc: 55545 Loss: 15368.197688
Test set: Average loss: 0.0016, Accuracy: 58109/60000 (97%)
Train Epoch: 8 Acc: 55627 Loss: 14952.036166
Test set: Average loss: 0.0016, Accuracy: 58117/60000 (97%)
Train Epoch: 9 Acc: 55792 Loss: 14707.972164
Test set: 

And that's it.